In [39]:
from stockfish import Stockfish
stockfish = Stockfish(depth=25) # default depth = 25
print(stockfish._stockfish_major_version)

16


In [40]:
import chess
import chess.pgn
import random
from pprint import pprint
import numpy as np
import os
import glob
import time

In [43]:
def checkEndCondition(board):
    if (board.is_checkmate()):
        print("Game over by checkmate")
        return True
    elif (board.is_stalemate()):
        print("Game over by stalemate")
        return True
    elif (board.is_insufficient_material()):
        print("Game over by insufficient material")
        return True
    elif (board.can_claim_threefold_repetition()):
        print("Game over by three-fold repetition")
        return True
    elif (board.can_claim_fifty_moves()):
        print("Game over by fifty moves")
        return True
    elif (board.can_claim_draw()):
        print("Game over by draw")
        return True
    return False

def findNextIdx():
    files = (glob.glob("data/*.npy"))
    if (len(files) == 0):
        return 1
    highestIdx = 0
    for f in files:
        file = f
        currIdx = file.split("movesAndPositions")[-1].split(".npy")[0]
        highestIdx = max(highestIdx, int(currIdx))

    return int(highestIdx) + 1

def saveData(moves, positions, number = 0):
    moves = np.array(moves).reshape(-1, 1)
    positions = np.array(positions).reshape(-1, 1)
    movesAndPositions = np.concatenate((moves, positions), axis = 1)
    nextIdx = findNextIdx()
    if number:
        np.save(f"data/movesAndPositions{number}.npy", movesAndPositions)
    else:
        np.save(f"data/movesAndPositions{nextIdx}.npy", movesAndPositions)
    print("Saved successfully")

def runGame(numMoves, filename = "movesAndPositions1.npy"):
    testing = np.load(f"data/{filename}")
    moves = testing[:, 0]
    if (numMoves > len(moves)):
        print("Must enter a lower number of moves than maximum game length.")
        print("Game length here is: ", len(moves))
        return
    testBoard = chess.Board()
    for i in range(numMoves):
        move = moves[i]
        testBoard.push_san(move)
    return testBoard

def mineGames(numGames : int):
    MAX_MOVES = 500

    for i in range(1, numGames + 1):
        print("-----Game ", i, "-----")
        currentGameMoves = []
        currentGamePositions = []
        board = chess.Board()
        game = chess.pgn.Game()
        node = game
        game.headers["Event"] = f"Game{i}"
        stockfish.set_position([])

        for j in range(MAX_MOVES):
            move = stockfish.get_best_move_time(2000)
            currentGamePositions.append(stockfish.get_fen_position())
            node = node.add_variation(chess.Move.from_uci(move))
            board.push_san(move)
            currentGameMoves.append(move)
            stockfish.set_position(currentGameMoves)
            if (checkEndCondition(board)):
                break
        saveData(currentGameMoves, currentGamePositions, i)
        print(game, file=open(f"data/game{i}.pgn", "w"), end = "\n\n")


In [44]:
mineGames(5)

------Game:  1
Game over by three-fold repetition
Saved successfully
------Game:  2
Game over by three-fold repetition
Saved successfully
------Game:  3


KeyboardInterrupt: 